In [7]:
search_for = 120
start_from = 133819999
threads = 5
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004406452178955078
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 875
all cases: 1116151311
type: [1, 1, 1, 1, 120] 120
cases of this type: 207360000
50000 0.19703026166064075
100000 0.18780150764791434
150000 0.1874708470586459
200000 0.19596169935930305
250000 0.18845469826302397
300000 0.18759788321295975
350000 0.18723329555371868
400000 0.20374005567467343
450000 0.20558704175205628
500000 0.197827117053922
550000 0.2501276666794777
600000 0.19190784715289275
650000 0.19124469782151776
700000 0.18709625915138328
750000 0.18704650531762837
800000 0.19308633406478692
850000 0.1943052893689341
900000 0.19659215274535816
950000 0.19436615586984557
1000000 0.19453764414158795
1050000 0.196363361667947
1100000 0.19605769227611491
1150000 0.19623498677570556
1200000 0.19686269046231508
1250000 0.19911327423792813
1300000 0.202112328922179
1350000 0.19715219953429303
1400000 0.21877320971174508
1450000 0.19495

14650000 0.20557912485659785
14700000 0.19441047760992053
14750000 0.1898952384113431
14800000 0.20592749796270396
14850000 0.1852509180578192
14900000 0.18602872516986263
14950000 0.18520346649873787
15000000 0.20562902613484063
15050000 0.18814586589459978
15100000 0.1859045197113249
15150000 0.19139106016111374
15200000 0.18670134590378074
15250000 0.19759725842341846
15300000 0.19536026139760018
15350000 0.20697507288597958
15400000 0.18731769645483756
15450000 0.188251011476175
15500000 0.19471683906329076
15550000 0.198345201773164
15600000 0.19416124465377332
15650000 0.19369763163500628
15700000 0.2077226060372061
15750000 0.20099102124145032
15800000 0.19789205697504414
15850000 0.19378776645799212
15900000 0.19402525000122786
15950000 0.19759402826244565
16000000 0.19784880207433037
16050000 0.21131540963720083
16100000 0.195458707695663
16150000 0.1944007806250453
16200000 0.2108116347857912
16250000 0.20160627778386145
16300000 0.1970693616013063
16350000 0.1968506322393695

28950000 0.19435922217599153
29000000 0.19125431865200335
29050000 0.19137799493961732
29100000 0.19928981457221506
29150000 0.19140790341557662
29200000 0.2131662078141782
29250000 0.195961191587166
29300000 0.19778764952674438
29350000 0.20259662430661124
29400000 0.19108318789974452
29450000 0.20190578870851728
29500000 0.20021393208243185
29550000 0.22327598092206719
29600000 0.1906292179555575
29650000 0.24186472936723497
29700000 0.2128278284457445
29750000 0.20140252363532252
29800000 0.19327340575178198
29850000 0.19011772807934682
29900000 0.19946831696801187
29950000 0.1995497751691461
30000000 0.1902526058363557
30050000 0.1903057072254631
30100000 0.19440250154142913
30150000 0.19519908945123352
30200000 0.20617080425007342
30250000 0.20813469768837556
30300000 0.1947535940954407
30350000 0.19638704567268292
30400000 0.190590969712068
30450000 0.2176264080704093
30500000 0.19877718822446397
30550000 0.18846382224232092
30600000 0.1997452537503481
30650000 0.1981372219505124

43350000 0.19232632298102376
43400000 0.18811288583540783
43450000 0.20212642762427593
43500000 0.1932982385328889
43550000 0.1921214033301923
43600000 0.19826668304134873
43650000 0.19278963131167096
43700000 0.19802375648728077
43750000 0.1936921050389078
43800000 0.1886279403529048
43850000 0.18787632096669146
43900000 0.18878646097537544
43950000 0.1898456343684554
44000000 0.19279772213297686
44050000 0.19107331143524114
44100000 0.1870813478005449
44150000 0.18620363330921463
44200000 0.18995187314286893
44250000 0.1869969604666869
44300000 0.18650837305661044
44350000 0.18716265021205877
44400000 0.191458102813375
44450000 0.1918281824897342
44500000 0.19170141816733016
44550000 0.1883299461321354
44600000 0.1885397342186742
44650000 0.18865015408826136
44700000 0.20929479875793058
44750000 0.20234268655553525
44800000 0.18800929366625546
44850000 0.20099988341748715
44900000 0.1934020998635848
44950000 0.18930676422681014
45000000 0.18822815964983308
45050000 0.1947029454430182

57650000 0.18983700011930335
57700000 0.18784564840390547
57750000 0.19207472750890253
57800000 0.1871031515852213
57850000 0.19665158366834853
57900000 0.186004758832705
57950000 0.1865860601535373
58000000 0.21014187138896917
58050000 0.18706736377482416
58100000 0.18724211256871884
58150000 0.18669934095182153
58200000 0.18640885504851737
58250000 0.18687926947293945
58300000 0.18661262082114882
58350000 0.20145274572490451
58400000 0.18938865789711076
58450000 0.18673186121948163
58500000 0.18767845429759425
58550000 0.18744824863304163
58600000 0.1849866259504159
58650000 0.18307011173179152
58700000 0.1957751075360841
58750000 0.18370266855210993
58800000 0.18349090776613952
58850000 0.1835508527060694
58900000 0.18671963420468568
58950000 0.1909453518743634
59000000 0.18607046181748177
59050000 0.20532675259507102
59100000 0.18599125564527513
59150000 0.18608186145136224
59200000 0.18957581503714854
59250000 0.18626304485067127
59300000 0.1863830192924804
59350000 0.185604320409

...

best so far: 0
type: [2, 3, 4, 5, 1] 120
cases of this type: 6000
best so far: 0
type: [2, 3, 5, 1, 4] 120
cases of this type: 19200
1101700000 0.17692191660570966
best so far: 0
type: [2, 3, 5, 2, 2] 120
cases of this type: 9600
best so far: 0
type: [2, 3, 5, 4, 1] 120
cases of this type: 4800
best so far: 0
type: [2, 3, 10, 1, 2] 120
cases of this type: 4800
best so far: 0
type: [2, 3, 10, 2, 1] 120
cases of this type: 2400
best so far: 0
type: [2, 3, 20, 1, 1] 120
cases of this type: 1200
best so far: 0
type: [2, 4, 1, 1, 15] 120
cases of this type: 202500
1101750000 0.2562077488110622
1101800000 0.38145140123563215
1101850000 0.2151407453244103
1101900000 0.36758263089386223
best so far: 0
type: [2, 4, 1, 3, 5] 120
cases of this type: 67500
1101950000 0.34324314780876897
1102000000 0.15849725368896325
best so far: 0
type: [2, 4, 1, 5, 3] 120
cases of this type: 40500
best so far: 0
type: [2, 4, 1, 15, 1] 120
cases of this type: 13500
1102050000 0.20705099108666977
best so far: 0
ty

1107300000 0.20845128563090565
1107350000 0.20745993188777498
1107400000 0.20406629789573486
1107450000 0.24762221291617156
1107500000 0.22685964904426334
1107550000 0.19867775160441928
1107600000 0.1729552406228026
best so far: 0
type: [3, 1, 1, 8, 5] 120
cases of this type: 320000
1107650000 0.18891608543486596
1107700000 0.23152511499972078
1107750000 0.21717596457668545
1107800000 0.21399748607220648
1107850000 0.20742214945485463
1107900000 0.21974852390721245
best so far: 0
type: [3, 1, 1, 10, 4] 120
cases of this type: 256000
1107950000 0.19104093511499828
1108000000 0.1404170472374916
1108050000 0.18128403086977798
1108100000 0.249688984178461
1108150000 0.20984945115933681
best so far: 0
type: [3, 1, 1, 20, 2] 120
cases of this type: 128000
1108200000 0.14139047053831816
1108250000 0.1026954590352363
1108300000 0.10434165581352976
best so far: 0
type: [3, 1, 1, 40, 1] 120
cases of this type: 64000
1108350000 0.08834970546871423
best so far: 0
type: [3, 1, 2, 1, 20] 120
cases o

1112150000 0.13796220168959034
1112200000 0.08761706388653913
1112250000 0.08049249746747016
1112300000 0.05961762772276402
1112350000 0.04495915241298941
1112400000 0.05797515851535002
1112450000 0.07705000113112397
1112500000 0.08676546842066846
best so far: 0
type: [4, 1, 1, 3, 10] 120
cases of this type: 270000
1112550000 0.07859083126943905
1112600000 0.08292252051575184
1112650000 0.07641887539202506
1112700000 0.04978596878138781
1112750000 0.08238310555637146
best so far: 0
type: [4, 1, 1, 5, 6] 120
cases of this type: 162000
1112800000 0.042083275344811544
1112850000 0.06532067604433694
1112900000 0.06556634156207775
1112950000 0.0638764767381615
best so far: 0
type: [4, 1, 1, 6, 5] 120
cases of this type: 135000
1113000000 0.0637679031649073
1113050000 0.06416030807418426
best so far: 0
type: [4, 1, 1, 10, 3] 120
cases of this type: 81000
1113100000 0.05265752578331232
1113150000 0.042685736411511896
best so far: 0
type: [4, 1, 1, 15, 2] 120
cases of this type: 54000
11132000

best so far: 0
type: [5, 2, 1, 2, 6] 120
cases of this type: 20736
best so far: 0
type: [5, 2, 1, 3, 4] 120
cases of this type: 13824
1115350000 0.008169836608331734
best so far: 0
type: [5, 2, 1, 4, 3] 120
cases of this type: 10368
best so far: 0
type: [5, 2, 1, 6, 2] 120
cases of this type: 6912
best so far: 0
type: [5, 2, 1, 12, 1] 120
cases of this type: 3456
best so far: 0
type: [5, 2, 2, 1, 6] 120
cases of this type: 10368
best so far: 0
type: [5, 2, 2, 2, 3] 120
cases of this type: 5184
best so far: 0
type: [5, 2, 2, 3, 2] 120
cases of this type: 3456
best so far: 0
type: [5, 2, 2, 6, 1] 120
cases of this type: 1728
best so far: 0
type: [5, 2, 3, 1, 4] 120
cases of this type: 4608
1115400000 0.009069809653782844
best so far: 0
type: [5, 2, 3, 2, 2] 120
cases of this type: 2304
best so far: 0
type: [5, 2, 3, 4, 1] 120
cases of this type: 1152
best so far: 0
type: [5, 2, 4, 1, 3] 120
cases of this type: 2592
best so far: 0
type: [5, 2, 4, 3, 1] 120
cases of this type: 864
best so 

best so far: 0
type: [10, 1, 3, 4, 1] 120
cases of this type: 576
best so far: 0
type: [10, 1, 4, 1, 3] 120
cases of this type: 1296
1116100000 0.0008006846731040213
best so far: 0
type: [10, 1, 4, 3, 1] 120
cases of this type: 432
best so far: 0
type: [10, 1, 6, 1, 2] 120
cases of this type: 576
best so far: 0
type: [10, 1, 6, 2, 1] 120
cases of this type: 288
best so far: 0
type: [10, 1, 12, 1, 1] 120
cases of this type: 144
best so far: 0
type: [10, 2, 1, 1, 6] 120
cases of this type: 2592
best so far: 0
type: [10, 2, 1, 2, 3] 120
cases of this type: 1296
best so far: 0
type: [10, 2, 1, 3, 2] 120
cases of this type: 864
best so far: 0
type: [10, 2, 1, 6, 1] 120
cases of this type: 432
best so far: 0
type: [10, 2, 2, 1, 3] 120
cases of this type: 648
best so far: 0
type: [10, 2, 2, 3, 1] 120
cases of this type: 216
best so far: 0
type: [10, 2, 3, 1, 2] 120
cases of this type: 288
best so far: 0
type: [10, 2, 3, 2, 1] 120
cases of this type: 144
best so far: 0
type: [10, 2, 6, 1, 1] 1